In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns

In [4]:
df = pd.read_excel('/content/listings_Excel_Python.xlsx')# considerando apenas as top 5 cidades em quantidade de imóveis (Copacabana, Barra da Tijuca, Ipanema, Jacarepaguá e
# Recreio dos Bandeirantes). Considerado também as medidas com maior correlação (vide arquivo 'corr_export_full.xlsx') e analisadas no Power BI.

In [5]:
df.head()

,neighbourhood_cleansed,accommodates,bathrooms,bedrooms,price,availability_30,availability_60,availability_90
0,Copacabana,5,1.0,2.0,350,6,26,45
1,Ipanema,2,1.0,1.0,100,5,25,55
2,Copacabana,2,1.0,1.0,266,21,51,81
3,Ipanema,2,1.0,1.0,624,29,59,82
4,Ipanema,5,3.0,3.0,1899,28,58,88


In [6]:
df.groupby('neighbourhood_cleansed')['price'].describe() # desconsiderado da base imóveis com aluguel = 0. Alto desvio padrão no Price em todos os bairros

,count,mean,std,min,25%,50%,75%,max
neighbourhood_cleansed,,,,,,,,
Barra da Tijuca,3017.0,1202.267153,6552.728475,53.0,300.0,491.0,950.0,281729.0
Copacabana,8973.0,1019.132509,9942.596189,50.0,210.0,303.0,527.0,500000.0
Ipanema,2954.0,971.706838,3226.566044,59.0,329.0,499.0,850.0,129487.0
Jacarepaguá,1533.0,737.114155,2749.542064,35.0,229.0,400.0,800.0,100000.0
Recreio dos Bandeirantes,1511.0,1069.849107,12908.044271,53.0,250.0,430.0,791.5,500560.0


In [7]:
df.groupby('neighbourhood_cleansed')['accommodates'].describe() # desconsiderado da base imóveis com aluguel = 0. Baixo desvio padrão em Acomodações em todos os bairros, boa medida para fazer correlação

,count,mean,std,min,25%,50%,75%,max
neighbourhood_cleansed,,,,,,,,
Barra da Tijuca,3017.0,4.457739,2.640286,1.0,3.0,4.0,6.0,16.0
Copacabana,8973.0,4.142985,2.156534,1.0,2.0,4.0,5.0,16.0
Ipanema,2954.0,4.111036,1.901893,1.0,3.0,4.0,5.0,16.0
Jacarepaguá,1533.0,3.933464,1.946949,1.0,2.0,4.0,5.0,16.0
Recreio dos Bandeirantes,1511.0,4.814030,2.523891,1.0,3.0,4.0,6.0,16.0


In [8]:
df.groupby('neighbourhood_cleansed')['bathrooms'].describe() # desconsiderado da base imóveis com aluguel = 0. Baixo desvio padrão em Banheiros em todos os bairros, boa medida para fazer correlação

,count,mean,std,min,25%,50%,75%,max
neighbourhood_cleansed,,,,,,,,
Barra da Tijuca,3015.0,1.834826,1.346398,0.0,1.0,1.0,2.0,15.0
Copacabana,8964.0,1.380968,0.711079,0.0,1.0,1.0,2.0,8.0
Ipanema,2948.0,1.648575,0.837273,0.0,1.0,2.0,2.0,15.0
Jacarepaguá,1532.0,1.492820,0.734038,0.0,1.0,1.0,2.0,12.0
Recreio dos Bandeirantes,1511.0,1.937790,1.185495,0.0,1.0,2.0,2.0,15.0


In [9]:
df.groupby('neighbourhood_cleansed')['bedrooms'].describe() # desconsiderado da base imóveis com aluguel = 0. Baixo desvio padrão em Quartos em todos os bairros, boa medida para fazer correlação

,count,mean,std,min,25%,50%,75%,max
neighbourhood_cleansed,,,,,,,,
Barra da Tijuca,2937.0,1.848485,1.143572,1.0,1.0,2.0,2.0,11.0
Copacabana,8417.0,1.555186,0.866205,1.0,1.0,1.0,2.0,8.0
Ipanema,2861.0,1.802167,0.879850,1.0,1.0,2.0,2.0,9.0
Jacarepaguá,1518.0,1.685112,0.786958,1.0,1.0,2.0,2.0,11.0
Recreio dos Bandeirantes,1488.0,1.959005,1.034216,1.0,1.0,2.0,3.0,15.0


In [12]:
df.corr() #podemos notar que há grande correlação entre Acomodações, Banheiros e Quartos entre si (acima dos 60%). Já com o Aluguel elas variam entre 0.05~0.06. Já as disponibilidades tem
# baixa correlação com o Aluguel, sendo essas descartadas a partir daqui.

<ipython-input-12-f2588a53c498>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr() #podemos notar que há grande correlação entre Acomodações, Banheiros e Quartos entre si (acima dos 60%). Já com o Aluguel elas variam entre 0.05~0.06. Já as disponibilidades tem


,accommodates,bathrooms,bedrooms,price,availability_30,availability_60,availability_90
accommodates,1.000000,0.631452,0.759716,0.053882,0.041191,0.042229,0.039726
bathrooms,0.631452,1.000000,0.769972,0.062790,0.081481,0.066607,0.057972
bedrooms,0.759716,0.769972,1.000000,0.056269,0.054813,0.043599,0.035508
price,0.053882,0.062790,0.056269,1.000000,-0.000705,-0.004543,-0.006747
availability_30,0.041191,0.081481,0.054813,-0.000705,1.000000,0.931695,0.866152
availability_60,0.042229,0.066607,0.043599,-0.004543,0.931695,1.000000,0.971174
availability_90,0.039726,0.057972,0.035508,-0.006747,0.866152,0.971174,1.000000


In [17]:
imov_copacabana = ['Copacabana']
df.query('@imov_copacabana in neighbourhood_cleansed').corr() # Nota-se um correlação de Acomodações, banheiros e Quartos baixa com Aluguel no bairro de Copacabana

<ipython-input-17-17f5c3440e85>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.query('@imov_copacabana in neighbourhood_cleansed').corr()


,accommodates,bathrooms,bedrooms,price,availability_30,availability_60,availability_90
accommodates,1.000000,0.601262,0.749092,0.041294,0.025360,0.032093,0.033000
bathrooms,0.601262,1.000000,0.750812,0.045001,0.036122,0.027536,0.023306
bedrooms,0.749092,0.750812,1.000000,0.046791,0.018288,0.016339,0.011925
price,0.041294,0.045001,0.046791,1.000000,-0.002765,-0.004211,-0.005954
availability_30,0.025360,0.036122,0.018288,-0.002765,1.000000,0.921186,0.849032
availability_60,0.032093,0.027536,0.016339,-0.004211,0.921186,1.000000,0.968517
availability_90,0.033000,0.023306,0.011925,-0.005954,0.849032,0.968517,1.000000


In [18]:
imov_Ipanema = ['Ipanema']
df.query('@imov_Ipanema in neighbourhood_cleansed').corr() # Nota-se um correlação de Acomodações, banheiros e Quartos maior com Aluguel no bairro de Ipanema

<ipython-input-18-a10a3d52ab88>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.query('@imov_Ipanema in neighbourhood_cleansed').corr()


,accommodates,bathrooms,bedrooms,price,availability_30,availability_60,availability_90
accommodates,1.000000,0.573285,0.774391,0.149994,0.014283,0.003543,-0.010361
bathrooms,0.573285,1.000000,0.681250,0.144520,0.039516,0.012161,-0.003759
bedrooms,0.774391,0.681250,1.000000,0.151131,0.012345,-0.011611,-0.030856
price,0.149994,0.144520,0.151131,1.000000,0.036798,0.019291,0.005510
availability_30,0.014283,0.039516,0.012345,0.036798,1.000000,0.922579,0.851802
availability_60,0.003543,0.012161,-0.011611,0.019291,0.922579,1.000000,0.967497
availability_90,-0.010361,-0.003759,-0.030856,0.005510,0.851802,0.967497,1.000000


In [19]:
imov_Barra_Tijuca = ['Barra da Tijuca']
df.query('@imov_Barra_Tijuca in neighbourhood_cleansed').corr() # Nota-se um correlação de Acomodações, banheiros e Quartos maior com Aluguel no bairro de Barra da Tijuca

<ipython-input-19-d0a3ac0061fb>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.query('@imov_Barra_Tijuca in neighbourhood_cleansed').corr()


,accommodates,bathrooms,bedrooms,price,availability_30,availability_60,availability_90
accommodates,1.000000,0.745352,0.831025,0.126098,0.039584,0.032866,0.026883
bathrooms,0.745352,1.000000,0.848855,0.145784,0.102620,0.082025,0.067508
bedrooms,0.831025,0.848855,1.000000,0.122013,0.090477,0.070192,0.058181
price,0.126098,0.145784,0.122013,1.000000,0.007908,-0.001561,0.002278
availability_30,0.039584,0.102620,0.090477,0.007908,1.000000,0.940258,0.877569
availability_60,0.032866,0.082025,0.070192,-0.001561,0.940258,1.000000,0.974874
availability_90,0.026883,0.067508,0.058181,0.002278,0.877569,0.974874,1.000000


In [20]:
imov_Jacarepagua = ['Jacarepaguá']
df.query('@imov_Jacarepagua in neighbourhood_cleansed').corr() # Nota-se um correlação de Acomodações, Banheiros e Quartos baixa com Aluguel no bairro de Jacarepaguá

<ipython-input-20-fad556298585>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.query('@imov_Jacarepagua in neighbourhood_cleansed').corr()


,accommodates,bathrooms,bedrooms,price,availability_30,availability_60,availability_90
accommodates,1.000000,0.577365,0.696470,0.066315,0.084165,0.087049,0.082212
bathrooms,0.577365,1.000000,0.715663,0.064060,0.084000,0.086461,0.077919
bedrooms,0.696470,0.715663,1.000000,0.079091,0.060984,0.057853,0.052927
price,0.066315,0.064060,0.079091,1.000000,0.065378,0.059919,0.051003
availability_30,0.084165,0.084000,0.060984,0.065378,1.000000,0.966819,0.927863
availability_60,0.087049,0.086461,0.057853,0.059919,0.966819,1.000000,0.979367
availability_90,0.082212,0.077919,0.052927,0.051003,0.927863,0.979367,1.000000


In [21]:
imov_Recreio_Bandeirantes = ['Recreio dos Bandeirantes']
df.query('@imov_Recreio_Bandeirantes in neighbourhood_cleansed').corr() # Nota-se um correlação de Acomodações, banheiros e Quartos baixa com Aluguel no bairro de Recreio dos Bandeirantes

<ipython-input-21-d1a2a57db3ef>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.query('@imov_Recreio_Bandeirantes in neighbourhood_cleansed').corr()


,accommodates,bathrooms,bedrooms,price,availability_30,availability_60,availability_90
accommodates,1.000000,0.638449,0.699066,0.007748,0.032022,0.040035,0.053172
bathrooms,0.638449,1.000000,0.782536,0.032431,0.069441,0.065263,0.069244
bedrooms,0.699066,0.782536,1.000000,0.005352,0.063738,0.059015,0.064558
price,0.007748,0.032431,0.005352,1.000000,-0.034273,-0.042114,-0.047032
availability_30,0.032022,0.069441,0.063738,-0.034273,1.000000,0.943540,0.879608
availability_60,0.040035,0.065263,0.059015,-0.042114,0.943540,1.000000,0.971650
availability_90,0.053172,0.069244,0.064558,-0.047032,0.879608,0.971650,1.000000


**Optado por seguir com a análise apenas com os bairros de Ipanema e Barra da Tijuca, devido uma maior correlação das metricas Acomodações, Banheiros e Quartos com Aluguel.**